### Model Training

[dataset](https://www.cellpose.org/dataset) from cellpose


In [ ]:
from cellpose import CellPoseModel
from evaluate import average_precision

import cv2
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import os
from PIL import Image

train_X, train_y = [], []
test_X, test_y = [], []

data_path = "./dataset/train/"
list_data = sorted(os.listdir(data_path))

print("Load train data")
for fpath in tqdm(list_data):
    if "img" in fpath:
        img = np.array(Image.open(os.path.join(data_path, fpath)).convert("L"))
        mask_fpath = fpath[:3] + "_masks.png"
        mask = np.array(Image.open(os.path.join(data_path, mask_fpath)))

        train_X.append(img)
        train_y.append(mask)

data_path = "./dataset/test/"
list_data = sorted(os.listdir(data_path))

print("Load test data")
for fpath in tqdm(list_data):
    if "img" in fpath:
        img = np.array(Image.open(os.path.join(data_path, fpath)).convert("L"))
        mask_fpath = fpath[:3] + "_masks.png"
        mask = np.array(Image.open(os.path.join(data_path, mask_fpath)))

        test_X.append(img)
        test_y.append(mask)

In [ ]:
model = CellPoseModel(device="cuda")
print("Start Training")
model.train(
    train_X,
    train_y,
    test_X,
    test_y,
    n_epochs=5,
    learning_rate=3e-4,
    save_path="./",
    model_name="test",
    batch_size=2,
    eval_batch_size=1,
    save_every=25,
)

### Evaluation

In [ ]:
masks = [model.predict(data, channels=[0,0], use_gpu=True)[0] for data in tqdm(test_X)]
ap = average_precision(test_y, masks)
print(ap[0][:, 0].mean())

### Model Inference

In [ ]:
from cellpose import CellPoseModel
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import os
from PIL import Image

def overlay(image, mask, alpha, resize=None):
    """Combines image and its segmentation mask into a single image.
    https://www.kaggle.com/code/purplejester/showing-samples-with-segmentation-mask-overlay

    Params:
        image: Training image. np.ndarray,
        mask: Segmentation mask. np.ndarray,
        color: Color for segmentation mask rendering.  tuple[int, int, int] = (255, 0, 0)
        alpha: Segmentation mask's transparency. float = 0.5,
        resize: If provided, both image and its mask are resized before blending them together.

    Returns:
        image_combined: The combined image. np.ndarray

    """
    color = list(np.random.choice(range(256), size=3))
    colored_mask = np.expand_dims(mask, 0).repeat(3, axis=0)
    colored_mask = np.moveaxis(colored_mask, 0, -1)
    masked = np.ma.MaskedArray(image, mask=colored_mask, fill_value=color)
    image_overlay = masked.filled()

    if resize is not None:
        image = cv2.resize(image.transpose(1, 2, 0), resize)
        image_overlay = cv2.resize(image_overlay.transpose(1, 2, 0), resize)

    image_combined = cv2.addWeighted(image, 1 - alpha, image_overlay, alpha, 0)

    return image_combined

model = CellPoseModel(pretrained_model='./models/model_ver1_epoch_100', device="cuda")
img = cv2.imread("./dataset/test/003_img.png")
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

print("Start Predicting")
masks, flows, styles = model.predict(gray_img, channels=[0,0], use_gpu=True, resample=False, tile=True, batch_infer=9)
img_overlay = overlay(np.array(img), masks, alpha=0.5)

plt.figure(figsize=(10,10))
plt.imshow(img_overlay)
plt.show()